<a href="https://colab.research.google.com/github/rtdatasci/Python/blob/main/stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install gradio
! pip install gnews
! pip install transformers
! pip install datetime
! pip install matplotlib
! pip install tensorflow
! pip install GoogleNews
! pip install pandas

In [3]:
import gradio as gr
from gnews import GNews
import pandas as pd
from transformers import pipeline
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import tensorflow as tf
from GoogleNews import GoogleNews

In [ ]:
SENTIMENT_ANALYSIS_MODEL = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
sentiment_analyzer = pipeline("sentiment-analysis", model=SENTIMENT_ANALYSIS_MODEL)

In [6]:
def fetch_articles(query):
    googlenews = GoogleNews(lang="en")
    googlenews.search(query)
    articles = googlenews.result()
    return articles

def analyze_article_sentiment(article):
    sentiment = sentiment_analyzer(article["desc"])[0]
    article["sentiment"] = sentiment
    return article

def extract_and_clean_titles(df):
    values_list = []
    if df.empty:
        return values_list
    for value in df['title']:
        index = value.find('-')
        extracted_value = value[:index] if index >= 0 else value
        cleaned_value = extracted_value.replace('...', '')
        values_list.append(cleaned_value)
    return values_list

def analyze_sentiments(values_list):
    prediction = []
    for news in values_list:
        sentiment = sentiment_analyzer(news)
        prediction.append(sentiment)
    return prediction

def calculate_weighted_average(predictions):
    weighted_avg = 0
    for i in predictions:
        if i[0]['label'] == 'positive':
            weighted_avg += 1 * i[0]['score']
        elif i[0]['label'] == 'negative':
            weighted_avg += -1 * i[0]['score']
    weighted_avg /= len(predictions)
    return weighted_avg

def sentiment_pie_chart(predictions, stock, output_path='sentiment_pie_chart.png'):
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    for item in predictions:
        label = item[0]['label']
        if label == 'positive':
            positive_count += 1
        elif label == 'negative':
            negative_count += 1
        elif label == 'neutral':
            neutral_count += 1
    labels = ['Positive', 'Negative', 'Neutral']
    sizes = [positive_count, negative_count, neutral_count]
    colors = ['#66BB6A', '#EF5350', '#42A5F5']
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, pctdistance=0.85)
    center_circle = plt.Circle((0, 0), 0.70, fc='white')
    fig.gca().add_artist(center_circle)
    ax.axis('equal')
    plt.title('Sentiment Analysis Results for ' + stock + ' Stock')
    plt.savefig(output_path)
    plt.close(fig)
    return output_path

def convert_to_dataframe(analyzed_articles):
    df = pd.DataFrame(analyzed_articles)
    df["Title"] = df.apply(lambda row: f'<a href="{row["link"]}" target="_blank">{row["title"]}</a>', axis=1)
    df["Description"] = df["desc"]
    df["Date"] = df["date"]

    def sentiment_badge(sentiment):
        colors = {"negative": "red", "neutral": "gray", "positive": "green"}
        color = colors.get(sentiment, "grey")
        return f'<span style="background-color: {color}; color: white; padding: 2px 6px; border-radius: 4px;">{sentiment}</span>'

    df["Sentiment"] = df["sentiment"].apply(lambda x: sentiment_badge(x["label"]))
    return df[["Sentiment", "Title", "Description", "Date"]]

def analyze_asset_sentiment(asset_name):
    articles = fetch_articles(asset_name)
    analyzed_articles = [analyze_article_sentiment(article) for article in articles]
    df = convert_to_dataframe(analyzed_articles)
    return df

def main(stock):
    articles = fetch_articles(stock + " stock")
    if not articles:
        return "Not enough data, please increase timeframe", None

    df = pd.DataFrame(articles)
    values_list = extract_and_clean_titles(df)
    predictions = analyze_sentiments(values_list)
    weighted_avg = calculate_weighted_average(predictions)
    pie_chart_path = sentiment_pie_chart(predictions, stock)

    if -0.10 <= weighted_avg <= 0.10:
        sentiment_summary = f'{weighted_avg:.2f} (Stagnant)'
    elif weighted_avg > 0.10:
        sentiment_summary = f'{weighted_avg:.2f} (Positive)'
    else:
        sentiment_summary = f'{weighted_avg:.2f} (Negative)'

    detailed_df = analyze_asset_sentiment(stock)
    return sentiment_summary, pie_chart_path, detailed_df

with gr.Blocks() as iface:
    gr.Markdown("# Trading Asset Sentiment Analysis")
    input_stock = gr.Textbox(label="Stock Name", lines=1, placeholder="Enter the name of the stock...")

    with gr.Row():
        analyze_button = gr.Button("Analyze Sentiment", size="sm")

    sentiment_summary = gr.Textbox(label="Sentiment Summary")
    sentiment_chart = gr.Image(label="Sentiment Distribution")
    articles_output = gr.Dataframe(headers=["Sentiment", "Title", "Description", "Date"], datatype=["markdown", "html", "markdown", "markdown"], wrap=False)

    analyze_button.click(main, inputs=[input_stock], outputs=[sentiment_summary, sentiment_chart, articles_output])

iface.queue().launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d2923d4db3b6f4b031.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
